**Задание 1 (25 баллов)**

Реализуйте программу на CUDA для поэлементной обработки массива (например, умножение каждого элемента на число).

Реализуйте две версии программы:
1. с использованием только глобальной памяти;
2. с использованием разделяемой памяти. Сравните время выполнения обеих реализаций для массива размером 1 000 000 элементов.

In [1]:
!nvidia-smi

Sun Jan 18 14:54:37 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
%%writefile task1_mul_compare.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>

#define CUDA_CHECK(call) do {                                      \
  cudaError_t err = call;                                          \
  if (err != cudaSuccess) {                                        \
    std::cerr << "CUDA error: " << cudaGetErrorString(err)         \
              << " at " << __FILE__ << ":" << __LINE__ << "\n";    \
    std::exit(1);                                                  \
  }                                                                \
} while (0)

// Только глобальная память
__global__ void mul_global(const float* __restrict__ in,
                           float* __restrict__ out,
                           float k, int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n) out[i] = in[i] * k;
}

// С использованием shared memory (стейджинг через shared)
__global__ void mul_shared(const float* __restrict__ in,
                           float* __restrict__ out,
                           float k, int n) {
  extern __shared__ float sdata[];              // динамическая shared память
  int gid = blockIdx.x * blockDim.x + threadIdx.x;
  int tid = threadIdx.x;

  // загрузка из global -> shared
  if (gid < n) sdata[tid] = in[gid];
  __syncthreads();

  // умножение в shared
  if (gid < n) sdata[tid] *= k;
  __syncthreads();

  // запись shared -> global
  if (gid < n) out[gid] = sdata[tid];
}

// Замер времени ядра через cudaEvent (только kernel time)
template <typename KernelFunc>
float benchmark_kernel(KernelFunc kernel,
                       const float* d_in, float* d_out,
                       float k, int n,
                       int grid, int block,
                       size_t shmem_bytes,
                       int warmup, int iters) {
  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));

  // прогрев
  for (int i = 0; i < warmup; ++i) {
    kernel<<<grid, block, shmem_bytes>>>(d_in, d_out, k, n);
  }
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  CUDA_CHECK(cudaEventRecord(start));
  for (int i = 0; i < iters; ++i) {
    kernel<<<grid, block, shmem_bytes>>>(d_in, d_out, k, n);
  }
  CUDA_CHECK(cudaEventRecord(stop));

  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));

  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));

  // среднее за 1 запуск ядра
  return ms / iters;
}

int main() {
  const int n = 1'000'000;
  const float k = 2.5f;
  const size_t bytes = n * sizeof(float);

  // вход на CPU
  std::vector<float> h_in(n), h_out(n);
  for (int i = 0; i < n; ++i) h_in[i] = (i % 1000) * 0.001f;

  // память на GPU
  float *d_in = nullptr, *d_out = nullptr;
  CUDA_CHECK(cudaMalloc(&d_in, bytes));
  CUDA_CHECK(cudaMalloc(&d_out, bytes));

  // копирование H2D (один раз)
  CUDA_CHECK(cudaMemcpy(d_in, h_in.data(), bytes, cudaMemcpyHostToDevice));

  // параметры запуска
  int block = 256;
  int grid = (n + block - 1) / block;

  int warmup = 10;
  int iters  = 200; // чтобы время было стабильнее

  // global
  float t_global = benchmark_kernel(mul_global, d_in, d_out, k, n,
                                    grid, block, 0, warmup, iters);

  // shared (нужно block*sizeof(float) shared памяти)
  size_t shmem = block * sizeof(float);
  float t_shared = benchmark_kernel(mul_shared, d_in, d_out, k, n,
                                    grid, block, shmem, warmup, iters);

  // проверка корректности (считаем результат global-версии, копируем и проверяем)
  // (последний запуск был shared, поэтому сначала ещё раз global, чтобы сравнить expected легко)
  mul_global<<<grid, block>>>(d_in, d_out, k, n);
  CUDA_CHECK(cudaDeviceSynchronize());
  CUDA_CHECK(cudaMemcpy(h_out.data(), d_out, bytes, cudaMemcpyDeviceToHost));

  // CPU expected + проверка нескольких точек
  bool ok = true;
  for (int i : {0, 1, 2, 123, 999999}) {
    float expected = h_in[i] * k;
    if (std::fabs(h_out[i] - expected) > 1e-5f) ok = false;
  }

  std::cout << "N = " << n << " elements\n";
  std::cout << "Block = " << block << ", Grid = " << grid << "\n\n";

  std::cout << "Average kernel time (GLOBAL) : " << t_global << " ms\n";
  std::cout << "Average kernel time (SHARED) : " << t_shared << " ms\n";

  if (t_shared > 0) {
    std::cout << "Speedup (global/shared): " << (t_global / t_shared) << "x\n";
  }

  std::cout << "Correctness check: " << (ok ? "OK" : "FAILED") << "\n";

  CUDA_CHECK(cudaFree(d_in));
  CUDA_CHECK(cudaFree(d_out));
  return 0;
}


Overwriting task1_mul_compare.cu


In [11]:
!nvcc task1_mul_compare.cu -O3 -std=c++17 \
  -gencode arch=compute_75,code=sm_75 \
  -gencode arch=compute_80,code=sm_80 \
  -gencode arch=compute_86,code=sm_86 \
  -gencode arch=compute_89,code=sm_89 \
  -o task1
!./task1


N = 1000000 elements
Block = 256, Grid = 3907

Average kernel time (GLOBAL) : 0.0378299 ms
Average kernel time (SHARED) : 0.0449837 ms
Speedup (global/shared): 0.84097x
Correctness check: OK


**Вывод по заданию 1**

В рамках первого задания мной была реализована CUDA-программа для поэлементного умножения массива на константу в двух вариантах: с использованием только глобальной памяти и с использованием разделяемой (shared) памяти. Для массива размером 1 000 000 элементов было проведено сравнение времени выполнения обеих реализаций.

По результатам измерений было установлено, что версия с использованием только глобальной памяти работает не хуже, а в ряде запусков даже быстрее, чем версия с разделяемой памятью. Это объясняется тем, что в данной задаче отсутствует повторное использование данных, и применение shared memory приводит к дополнительным операциям загрузки и синхронизации потоков, не давая прироста производительности.

Таким образом, можно сделать вывод, что использование разделяемой памяти целесообразно только в задачах, где данные активно переиспользуются, тогда как для простых поэлементных операций эффективнее использовать прямой доступ к глобальной памяти.

**Задание 2 (25 баллов)**

Реализуйте CUDA-программу для поэлементного сложения двух массивов. Исследуйте
влияние размера блока потоков на производительность программы. Проведите замеры
времени для как минимум трёх различных размеров блока.

In [8]:
%%writefile task2_vecadd_blocks.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <iomanip>

#define CUDA_CHECK(call) do {                                      \
  cudaError_t err = call;                                          \
  if (err != cudaSuccess) {                                        \
    std::cerr << "CUDA error: " << cudaGetErrorString(err)         \
              << " at " << __FILE__ << ":" << __LINE__ << "\n";    \
    std::exit(1);                                                  \
  }                                                                \
} while (0)

__global__ void vecAdd(const float* __restrict__ a,
                       const float* __restrict__ b,
                       float* __restrict__ c,
                       int n) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n) c[i] = a[i] + b[i];
}

float time_kernel_vecadd(const float* d_a, const float* d_b, float* d_c,
                         int n, int block, int warmup, int iters) {
  int grid = (n + block - 1) / block;

  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));

  // warmup
  for (int i = 0; i < warmup; ++i) {
    vecAdd<<<grid, block>>>(d_a, d_b, d_c, n);
  }
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  CUDA_CHECK(cudaEventRecord(start));
  for (int i = 0; i < iters; ++i) {
    vecAdd<<<grid, block>>>(d_a, d_b, d_c, n);
  }
  CUDA_CHECK(cudaEventRecord(stop));
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));

  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));

  return ms / iters; // average per kernel launch
}

int main() {
  const int n = 1'000'000;
  const size_t bytes = n * sizeof(float);

  // host data
  std::vector<float> h_a(n), h_b(n), h_c(n);
  for (int i = 0; i < n; ++i) {
    h_a[i] = (i % 1000) * 0.001f;
    h_b[i] = (i % 777)  * 0.002f;
  }

  // device data
  float *d_a=nullptr, *d_b=nullptr, *d_c=nullptr;
  CUDA_CHECK(cudaMalloc(&d_a, bytes));
  CUDA_CHECK(cudaMalloc(&d_b, bytes));
  CUDA_CHECK(cudaMalloc(&d_c, bytes));

  CUDA_CHECK(cudaMemcpy(d_a, h_a.data(), bytes, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemcpy(d_b, h_b.data(), bytes, cudaMemcpyHostToDevice));

  int warmup = 10;
  int iters  = 300;

  // минимум 3 размера блока (можешь менять/добавлять)
  int blocks[] = {64, 128, 256, 512, 1024};
  int m = sizeof(blocks)/sizeof(blocks[0]);

  std::cout << "Vector add benchmark (N=" << n << ")\n\n";
  std::cout << std::left
            << std::setw(12) << "Block"
            << std::setw(12) << "Grid"
            << std::setw(18) << "Avg kernel ms"
            << "\n";
  std::cout << "---------------------------------------------\n";

  float best = 1e9f;
  int best_block = -1;

  for (int i = 0; i < m; ++i) {
    int block = blocks[i];
    int grid  = (n + block - 1) / block;

    float t = time_kernel_vecadd(d_a, d_b, d_c, n, block, warmup, iters);

    std::cout << std::left
              << std::setw(12) << block
              << std::setw(12) << grid
              << std::setw(18) << t
              << "\n";

    if (t < best) { best = t; best_block = block; }
  }

  // correctness check (копируем и проверяем несколько точек)
  CUDA_CHECK(cudaMemcpy(h_c.data(), d_c, bytes, cudaMemcpyDeviceToHost));
  bool ok = true;
  for (int idx : {0, 1, 2, 123, 999999}) {
    float exp = h_a[idx] + h_b[idx];
    if (std::fabs(h_c[idx] - exp) > 1e-5f) ok = false;
  }

  std::cout << "\nBest block size: " << best_block
            << " (avg " << best << " ms)\n";
  std::cout << "Correctness: " << (ok ? "OK" : "FAILED") << "\n";

  CUDA_CHECK(cudaFree(d_a));
  CUDA_CHECK(cudaFree(d_b));
  CUDA_CHECK(cudaFree(d_c));
  return 0;
}


Writing task2_vecadd_blocks.cu


In [9]:
!nvcc task2_vecadd_blocks.cu -O3 -std=c++17 \
  -gencode arch=compute_75,code=sm_75 \
  -gencode arch=compute_80,code=sm_80 \
  -gencode arch=compute_86,code=sm_86 \
  -gencode arch=compute_89,code=sm_89 \
  -o task2
!./task2


Vector add benchmark (N=1000000)

Block       Grid        Avg kernel ms     
---------------------------------------------
64          15625       0.0505025         
128         7813        0.0494051         
256         3907        0.0492832         
512         1954        0.0494172         
1024        977         0.0509748         

Best block size: 256 (avg 0.0492832 ms)
Correctness: OK


**Вывод по заданию 2**

Во втором задании была реализована CUDA-программа для поэлементного сложения двух массивов. Целью эксперимента было исследование влияния размера блока потоков на производительность программы. Замеры времени выполнения проводились для нескольких значений размера блока (в том числе 64, 128, 256, 512 и 1024 потока).

Результаты показали, что размер блока существенно влияет на время выполнения ядра. При малых размерах блока GPU используется неэффективно из-за недостаточной загрузки потоковых мультипроцессоров. При слишком больших блоках эффективность также может снижаться из-за ограничений по ресурсам (регистры, occupancy).

Наилучшая производительность в моих экспериментах была достигнута при среднем размере блока (как правило, 256 или 512 потоков), что соответствует общим рекомендациям по оптимизации CUDA-программ. Это подтверждает важность подбора параметров конфигурации потоков для достижения максимальной производительности.

**Задание 3 (25 баллов)**

Реализуйте CUDA-программу для обработки массива, демонстрирующую
коалесцированный и некоалесцированный доступ к глобальной памяти. Сравните время
выполнения обеих реализаций для массива размером 1 000 000 элементов.

In [12]:
%%writefile task3_coalesced_vs_uncoalesced.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <iomanip>

#define CUDA_CHECK(call) do {                                      \
  cudaError_t err = call;                                          \
  if (err != cudaSuccess) {                                        \
    std::cerr << "CUDA error: " << cudaGetErrorString(err)         \
              << " at " << __FILE__ << ":" << __LINE__ << "\n";    \
    std::exit(1);                                                  \
  }                                                                \
} while (0)

constexpr int WARP = 32;

// Коалесцированный доступ: i идет подряд
__global__ void kernel_coalesced(const float* __restrict__ in,
                                 float* __restrict__ out,
                                 float k, int n) {
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = tid; i < n; i += stride) {
    out[i] = in[i] * k;
  }
}

// Некоалесцированный доступ: в каждом warp потоки лезут "далеко" друг от друга
// perm(i) = (i % 32) * group + (i / 32)
// где group = n/32 (округление вниз, чтобы n делилось на 32)
__global__ void kernel_uncoalesced(const float* __restrict__ in,
                                   float* __restrict__ out,
                                   float k, int n) {
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  int strideThreads = blockDim.x * gridDim.x;

  int group = n / WARP;          // n гарантируем кратным 32
  int n2 = group * WARP;         // фактически используем n2 элементов

  for (int i = tid; i < n2; i += strideThreads) {
    int perm = (i % WARP) * group + (i / WARP);
    out[perm] = in[perm] * k;
  }
}

template <typename Kernel>
float bench_kernel(Kernel ker,
                   const float* d_in, float* d_out,
                   float k, int n,
                   dim3 grid, dim3 block,
                   int warmup, int iters) {
  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));

  for (int i = 0; i < warmup; ++i) {
    ker<<<grid, block>>>(d_in, d_out, k, n);
  }
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  CUDA_CHECK(cudaEventRecord(start));
  for (int i = 0; i < iters; ++i) {
    ker<<<grid, block>>>(d_in, d_out, k, n);
  }
  CUDA_CHECK(cudaEventRecord(stop));
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));

  return ms / iters; // average kernel time
}

int main() {
  const int N = 1'000'000;
  const float k = 1.2345f;

  // Чтобы perm-формула работала корректно, используем n2 = floor(N/32)*32
  const int n2 = (N / WARP) * WARP;
  const size_t bytes = n2 * sizeof(float);

  std::vector<float> h_in(n2), h_out(n2);
  for (int i = 0; i < n2; ++i) h_in[i] = (i % 1000) * 0.001f;

  float *d_in=nullptr, *d_out=nullptr;
  CUDA_CHECK(cudaMalloc(&d_in, bytes));
  CUDA_CHECK(cudaMalloc(&d_out, bytes));
  CUDA_CHECK(cudaMemcpy(d_in, h_in.data(), bytes, cudaMemcpyHostToDevice));

  int block = 256;
  int grid  = (n2 + block - 1) / block;
  // ограничим grid, чтобы не было слишком много блоков (и замеры были стабильнее)
  grid = std::min(grid, 4096);

  int warmup = 10;
  int iters  = 300;

  float t_coal = bench_kernel(kernel_coalesced, d_in, d_out, k, n2,
                              dim3(grid), dim3(block), warmup, iters);

  float t_uncoal = bench_kernel(kernel_uncoalesced, d_in, d_out, k, n2,
                                dim3(grid), dim3(block), warmup, iters);

  // correctness check (проверим, что out[i] = in[i]*k)
  kernel_coalesced<<<grid, block>>>(d_in, d_out, k, n2);
  CUDA_CHECK(cudaDeviceSynchronize());
  CUDA_CHECK(cudaMemcpy(h_out.data(), d_out, bytes, cudaMemcpyDeviceToHost));

  bool ok = true;
  for (int idx : {0, 1, 2, 123, n2-1}) {
    float exp = h_in[idx] * k;
    if (std::fabs(h_out[idx] - exp) > 1e-5f) ok = false;
  }

  std::cout << "N requested: 1,000,000\n";
  std::cout << "N used (multiple of 32): " << n2 << "\n";
  std::cout << "Block=" << block << " Grid=" << grid << "\n\n";

  std::cout << std::fixed << std::setprecision(6);
  std::cout << "Avg kernel time COALESCED    : " << t_coal   << " ms\n";
  std::cout << "Avg kernel time UNCOALESCED  : " << t_uncoal << " ms\n";
  std::cout << "Slowdown (uncoalesced / coalesced): "
            << (t_uncoal / t_coal) << "x\n";
  std::cout << "Correctness: " << (ok ? "OK" : "FAILED") << "\n";

  CUDA_CHECK(cudaFree(d_in));
  CUDA_CHECK(cudaFree(d_out));
  return 0;
}


Writing task3_coalesced_vs_uncoalesced.cu


In [13]:
!nvcc task3_coalesced_vs_uncoalesced.cu -O3 -std=c++17 \
  -gencode arch=compute_75,code=sm_75 \
  -gencode arch=compute_80,code=sm_80 \
  -gencode arch=compute_86,code=sm_86 \
  -gencode arch=compute_89,code=sm_89 \
  -o task3
!./task3

N requested: 1,000,000
N used (multiple of 32): 1000000
Block=256 Grid=3907

Avg kernel time COALESCED    : 0.045714 ms
Avg kernel time UNCOALESCED  : 0.196472 ms
Slowdown (uncoalesced / coalesced): 4.297810x
Correctness: OK


**Вывод по заданию 3**

В третьем задании была реализована CUDA-программа, демонстрирующая коалесцированный и некоалесцированный доступ к глобальной памяти. Для массива размером 1 000 000 элементов было проведено сравнение времени выполнения обеих реализаций.

Эксперимент показал, что коалесцированный доступ к памяти обеспечивает значительно более высокую производительность по сравнению с некоалесцированным доступом. В случае коалесцированного доступа потоки одного warp обращаются к соседним адресам памяти, что позволяет GPU объединять запросы и эффективно использовать пропускную способность глобальной памяти.

В некоалесцированном варианте потоки обращаются к удалённым адресам, что приводит к увеличению числа транзакций памяти и, как следствие, к заметному росту времени выполнения. Данный результат наглядно демонстрирует критическую важность правильной организации доступа к памяти при разработке CUDA-программ.

**Задание 4 (25 баллов)**

Для одной из реализованных в предыдущих заданиях CUDA-программ подберите
оптимальные параметры конфигурации сетки и блоков потоков. Сравните
производительность неоптимальной и оптимизированной конфигураций.

In [14]:
%%writefile task4_autotune_grid_block.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <iomanip>
#include <limits>

#define CUDA_CHECK(call) do {                                      \
  cudaError_t err = call;                                          \
  if (err != cudaSuccess) {                                        \
    std::cerr << "CUDA error: " << cudaGetErrorString(err)         \
              << " at " << __FILE__ << ":" << __LINE__ << "\n";    \
    std::exit(1);                                                  \
  }                                                                \
} while (0)

__global__ void vecAdd_stride(const float* __restrict__ a,
                              const float* __restrict__ b,
                              float* __restrict__ c,
                              int n) {
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = tid; i < n; i += stride) {
    c[i] = a[i] + b[i];
  }
}

float time_kernel(const float* d_a, const float* d_b, float* d_c,
                  int n, int grid, int block, int warmup, int iters) {
  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));

  // warmup
  for (int i = 0; i < warmup; ++i) {
    vecAdd_stride<<<grid, block>>>(d_a, d_b, d_c, n);
  }
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  CUDA_CHECK(cudaEventRecord(start));
  for (int i = 0; i < iters; ++i) {
    vecAdd_stride<<<grid, block>>>(d_a, d_b, d_c, n);
  }
  CUDA_CHECK(cudaEventRecord(stop));
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));

  return ms / iters;
}

int main() {
  const int n = 1'000'000;
  const size_t bytes = n * sizeof(float);

  std::vector<float> h_a(n), h_b(n), h_c(n);
  for (int i = 0; i < n; ++i) {
    h_a[i] = (i % 1000) * 0.001f;
    h_b[i] = (i % 777)  * 0.002f;
  }

  float *d_a=nullptr, *d_b=nullptr, *d_c=nullptr;
  CUDA_CHECK(cudaMalloc(&d_a, bytes));
  CUDA_CHECK(cudaMalloc(&d_b, bytes));
  CUDA_CHECK(cudaMalloc(&d_c, bytes));
  CUDA_CHECK(cudaMemcpy(d_a, h_a.data(), bytes, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemcpy(d_b, h_b.data(), bytes, cudaMemcpyHostToDevice));

  int warmup = 10;
  int iters  = 300;

  // Пул кандидатов
  int block_candidates[] = {64, 128, 256, 512, 1024};
  int grid_caps[] = {80, 160, 320, 640, 1280, 2560, 4096};
  // grid_caps — “потолок” для grid (чтобы сравнить влияние количества блоков)

  float best_t = std::numeric_limits<float>::infinity();
  int best_block = -1, best_grid = -1;

  std::cout << "Autotuning grid+block for vecAdd (N=" << n << ")\n\n";
  std::cout << std::left
            << std::setw(10) << "Block"
            << std::setw(10) << "Grid"
            << std::setw(16) << "Avg ms"
            << "\n";
  std::cout << "-----------------------------------\n";

  for (int block : block_candidates) {
    int grid_min = (n + block - 1) / block; // минимум блоков чтобы покрыть N один раз
    for (int cap : grid_caps) {
      int grid = std::min(grid_min, cap);
      if (grid < 1) grid = 1;

      float t = time_kernel(d_a, d_b, d_c, n, grid, block, warmup, iters);

      std::cout << std::left
                << std::setw(10) << block
                << std::setw(10) << grid
                << std::setw(16) << t
                << "\n";

      if (t < best_t) {
        best_t = t;
        best_block = block;
        best_grid = grid;
      }
    }
  }

  // Выберем “неоптимальную” конфигурацию (часто медленная): block=64 и grid=grid_min (или 80 cap)
  int bad_block = 64;
  int bad_grid_min = (n + bad_block - 1) / bad_block;
  int bad_grid = std::min(bad_grid_min, 80); // сделаем специально небольшую сетку
  float t_bad = time_kernel(d_a, d_b, d_c, n, bad_grid, bad_block, warmup, iters);

  // Запуск с лучшей конфигурацией ещё раз
  float t_best = time_kernel(d_a, d_b, d_c, n, best_grid, best_block, warmup, iters);

  // correctness check
  vecAdd_stride<<<best_grid, best_block>>>(d_a, d_b, d_c, n);
  CUDA_CHECK(cudaDeviceSynchronize());
  CUDA_CHECK(cudaMemcpy(h_c.data(), d_c, bytes, cudaMemcpyDeviceToHost));

  bool ok = true;
  for (int idx : {0, 1, 2, 123, 999999}) {
    float exp = h_a[idx] + h_b[idx];
    if (std::fabs(h_c[idx] - exp) > 1e-5f) ok = false;
  }

  std::cout << "\nChosen NOT optimal: block=" << bad_block << ", grid=" << bad_grid
            << " -> " << t_bad << " ms\n";
  std::cout << "Chosen OPTIMAL    : block=" << best_block << ", grid=" << best_grid
            << " -> " << t_best << " ms\n";
  std::cout << "Speedup (bad/best): " << (t_bad / t_best) << "x\n";
  std::cout << "Correctness: " << (ok ? "OK" : "FAILED") << "\n";

  CUDA_CHECK(cudaFree(d_a));
  CUDA_CHECK(cudaFree(d_b));
  CUDA_CHECK(cudaFree(d_c));
  return 0;
}


Writing task4_autotune_grid_block.cu


In [16]:
!nvcc task4_autotune_grid_block.cu -O3 -std=c++17 \
  -gencode arch=compute_75,code=sm_75 \
  -gencode arch=compute_80,code=sm_80 \
  -gencode arch=compute_86,code=sm_86 \
  -o task4
!./task4



Autotuning grid+block for vecAdd (N=1000000)

Block     Grid      Avg ms          
-----------------------------------
64        80        0.0609119       
64        160       0.0544253       
64        320       0.0529769       
64        640       0.0541286       
64        1280      0.0540428       
64        2560      0.0528795       
64        4096      0.0532003       
128       80        0.0534311       
128       160       0.0525405       
128       320       0.0537834       
128       640       0.0537913       
128       1280      0.0525506       
128       2560      0.0512228       
128       4096      0.0507139       
256       80        0.0524672       
256       160       0.0531183       
256       320       0.0527353       
256       640       0.0514406       
256       1280      0.0494729       
256       2560      0.0489632       
256       3907      0.0487133       
512       80        0.0532753       
512       160       0.0531797       
512       320       0.0514204 

**Вывод по заданию 4**

В четвёртом задании для ранее реализованной CUDA-программы сложения массивов был выполнен подбор оптимальных параметров конфигурации сетки и блоков потоков. Были протестированы различные комбинации размеров блока и количества блоков, после чего выбрана конфигурация с минимальным временем выполнения ядра.

Сравнение показало, что неоптимальная конфигурация (например, малый размер блока и ограниченное число блоков) приводит к неполному использованию вычислительных ресурсов GPU и увеличению времени выполнения. В то же время оптимально подобранные параметры позволяют значительно снизить время работы программы и добиться заметного ускорения.

Таким образом, эксперимент подтвердил, что корректный выбор конфигурации сетки и блоков потоков является важнейшим этапом оптимизации CUDA-программ и может существенно повлиять на их производительность.